# 생성 모델을 위한 딥러닝
- Augmented Intelligence(확장된 지능)
  - AI가 사람의 능력을 증가시키는 도구로 사용
- 예술 창작의 대부분은 간단한 패턴 인식과 기교로 구성됨
  - 사람의 지각, 언어, 예술 작품 모두 통계적 구조
  - 딥러닝은 이런 구조, 즉 latent space(잠재 공간)를 학습하는 것

## 텍스트 생성

### 시퀀스 생성을 위한 딥러닝 모델의 간단한 역사

### 시퀀스 데이터를 어떻게 생성할까?
- 시퀀스 데이터를 생성하는 일반적인 방법
  - 이전 토큰을 입력으로 사용해서 시퀀스의 다음 1개 또는 몇 개의 토큰을 트랜스포머나 RNN으로 예측
- Language Model
  - 이전 토큰(단어 혹은 글자)들이 주어졌을 때 다음 토큰의 확률을 모델링할 수 있는 네트워크
  - 언어의 통계적 구조인 잠재 공간을 탐색
- 모델이 훈련하고 나면 해당 모델에서 샘플링 가능 : 새로운 시퀀스 생성
  - 초기 텍스트 문자열을 주입 (conditioning data)
  - 새로운 글자나 단어를 생성 (여러 개도 ㄱㅊ)
  - 생성된 출력은 다시 입력 데이터로 추가
  - 반복


### 샘플링 전략의 중요성
- Greedy sampling
  - 엔트로피가 최소
  - 반복적이고 예상 가능한 문자열
- Stochastic sampling
  - 무작위성(random) 주입
  - 엔트로피가 높음
    - 근데 엔트로피가 최대인 경우 : 완전히 무작위, 모든 단어의 확률이 같음 -> 별로 안 흥미로움 -> 적당히 softmax 출력을 사용하자
  - Softmax 출력, 실제 모델의 확률 분포에서 샘플링하는게 엔트로피가 최소, 최대 두 지점에 사이인거
- Softmax temperature 파라미터
  - 샘플링 과정에서 확률의 양을 조절


**다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기**

In [1]:
import numpy as np
# original_distribution : 전체 합이 1인 1D numpy array
# temperature : 출력 분포의 엔트로피 양을 결정
def reweight_distribution(original_distribution, temperature=0.5):
    # 원본 분포의 가중치를 변경
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    # 새로 바꾼 분포의 합이 1이 아닐 수도 있으니, 새로운 분포의 합으로 나눠줌
    return distribution / np.sum(distribution)

### 케라스를 사용한 텍스트 생성 모델 구현
- IMDB 영화 리뷰 데이터셋 사용
- 리뷰 생성 : 스타일과 주제를 모델링

#### 데이터 준비

**IMDB 영화 리뷰 데이터셋 다운로드하고 압축 풀기**

In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2025-02-10 05:59:14--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  51.4MB/s    in 1.6s    

2025-02-10 05:59:15 (51.4 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



**텍스트 파일(한 파일 = 한 샘플)에서 데이터셋 만들기**  
- `label_mode=None`

In [4]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 100006 files.


**`TextVectorization` 층 준비하기**

In [5]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 100
# 가장 많이 등장하는 15000개 단어만 사용
# 그 외는 OOV 토큰인 "[UNK]"으로 처리
vocab_size = 15000
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    # 정수 단어 인덱스의 시퀀스를 반환
    output_mode="int",
    # 길이가 100인 input / target
    # 근데 타겟은 한 스텝 차이나기 때문에 실제로 모델은 99개의 단어 시퀀스를 봄
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

**언어 모델링 데이터셋 만들기**  
- Input : Vectorized text
- Output : Previous text

In [7]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    # 시퀀스의 마지막 단어를 제외한 입력
    x = vectorized_sequences[:, :-1]
    # 시퀀스의 첫 단어를 제외한 타겟
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

#### 트랜스포머 기반의 시퀀스-투-시퀀스 모델
- 훈련 : 초기 몇 개의 단어가 주어지면 문장의 다음 단어에 대한 확률 분포를 예측하는 모델 훈련
  - 초기 문장을 주입
  - 다음 단어를 샘플링하여 해당 문장에 추가
  - 짧은 문단을 생성할 때까지 반복
- 문제점
  - N개의 단어로 예측을 만드는 방법을 학습하지만, N개보다 적은 단어로 예측을 시작할 수 있어야 함
  - 훈련에 사용하는 많은 시퀀스는 중복
- 해결 방법 : Seq-to-seq 모델 사용
  - 단어 N개의 시퀀스를 모델에 주입, 한 스텝 다음의 시퀀스를 예측
  - Causal masking을 사용하여 어떤 인덱스 i에서 모델은 0에서 i까지 단어만 사용해서 i+1번째 단어를 예측
  - 즉, 1 <= i <= N 인 단어의 시퀀스에서 다음 단어를 예측
  - 따라서, 생성 단계에서는 하나의 단어만 모델에 전달해도 다음 단어에 대한 확률 분포를 만들 수 있음
- Seq-to-Seq
  - 소스 시퀀스를 인코더에 주입
  - 인코딩된 시퀀스와 타겟 시퀀스를 디코더로 전달
  - 한 스텝 후의 타겟 시퀀스 예측
- 근데 텍스트 생성에서는 소스 시퀀스가 없음
  - 과거 토큰이 주어지면 타겟 시퀀스에 있는 다음 토큰을 예측하는 것뿐
  - 따라서 디코더만 사용해서 수행
  - Causal padding으로 인해 디코더는 단어 0부터 N만 사용해서 N + 1를 예측 가능
- 11장에서 만든 `PositionalEmbedding`과 `TransformerDecoder`를 재사용

In [13]:
import tensorflow as tf
from tensorflow.keras import layers

class NotEqualLayer(keras.Layer):
    def call(self, x):
        return tf.math.not_equal(x, 0)

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return NotEqualLayer()(inputs)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    # def get_causal_attention_mask(self, inputs):
    #     input_shape = tf.shape(inputs)
    #     batch_size, sequence_length = input_shape[0], input_shape[1]
    #     i = tf.range(sequence_length)[:, tf.newaxis]
    #     j = tf.range(sequence_length)
    #     mask = tf.cast(i >= j, dtype="int32")
    #     mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
    #     mult = tf.concat(
    #         [tf.expand_dims(batch_size, -1),
    #          tf.constant([1, 1], dtype=tf.int32)], axis=0)
    #     return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        # causal_mask = self.get_causal_attention_mask(inputs)
        # if mask is not None:
        #     padding_mask = tf.cast(
        #         mask[:, tf.newaxis, :], dtype="int32")
        #     padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            use_causal_mask=True)
            # attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs
            # attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**간단한 트랜스포머 기반 언어 모델**

In [14]:
from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
# 출력 시퀀스 타임스텝마다 가능한 어휘 사전의 단어에 대해
# 소프트맥스 확률을 계산
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

### 가변 온도 샘플링을 사용한 텍스트 생성 콜백
- 콜백을 사용하여 에포크가 끝날 때마다 다양한 온도로 텍스트를 생성
- 모델이 수렴하면서 생성된 텍스트가 어떻게 발전하는지 온도가 샘플링 전략에 미치는 영향 확인

**텍스트 생성 콜백**

In [15]:
import numpy as np

# 단어 인덱스를 문자열로 매핑하는 딕셔너리
# 텍스트 디코딩에 사용
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

# 어떤 확률 분포에 대한 가변 온도 샘플링을 구현
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,             # 텍스트 생성을 위한 시작 문장
                 generate_length,    # 생성할 단어 개수
                 model_input_length,
                 temperatures=(1.,), # 샘플링에 사용할 온도 범위
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            # 시작 단어에서부터 텍스트 생성
            sentence = self.prompt
            for i in range(self.generate_length):
                # 현재 시퀀스를 모델에 주입
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                # 마지막 타임스텝의 예측을 추출하여 다음 단어를 샘플링
                next_token = sample_next(predictions[0, i, :], temperature)
                sampled_token = tokens_index[next_token]
                # 새로운 단어를 현재 시퀀스에 추가하고 반복
                sentence += " " + sampled_token
            print(sentence)

prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    # 텍스트 샘플링에 다양한 온도를 사용하여 텍스트 생성에 미치는 온도의 영향을 확인
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

**언어 모델 훈련하기**

In [16]:
# 코랩에서 정상 실행만 확인하기 위해 에포크 횟수를 200에서 10으로 줄입니다
model.fit(lm_dataset, epochs=10,  # 200
          callbacks=[text_gen_callback])

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step - loss: 6.3848== Generating with temperature 0.2
This movie is is a a very [UNK] good [UNK] movie [UNK] is [UNK] a [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]
== Generating with temperature 0.5
This movie is is a a decent child movie and is i a was couple not of so the much best like [UNK] [UNK] the [UNK] [UNK] and and the they [UNK] are [UNK] all and the [UNK] movie [UNK] is are a the little [UNK] girl woman who who wants is
== Generating with temperature 0.7
This movie is is one [UNK] of and an 1962 cheated is for a [UNK] [UNK] of [UNK] a [UNK] great [UNK] film and you [UNK] ever cinematography made and by uninspired king for than [UNK] the [UNK] music the style [UNK] was is not not only seen thing in about a
== Generating with temperature 1.0
This movie w

KeyboardInterrupt: 

### 정리
- 이전 토큰이 주어지면 다음 토큰들을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성
  - 언어 모델 : 단어 / 글자 단위 모두 가능
- 모델이 만든 출력에 집중하는 것(엔트로피가 낮음)과 무작위성을 주입하는 것(엔트로피가 높음) 사이에 균형이 중요
  - 소프트맥스에 temperature 개념을 도입 (맥스웰-볼츠만 분포 개념)

![high_temp.png](./images/high_temp.png)
![low_temp.png](./images/low_temp.png)
- fn1(x) = log(x)/a , fn2(x) = exp(fn1)
  - `a`를 크게 하면 함수의 변화가 완만함 (첫 번째 그림)
  - `a`를 작게 하면 함수의 변화가 급격함 (두 번째 그림)
- a를 temperature 파라미터로 치환하여 생각
```python
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)
```
  - `temperature` 값이 커지면, 즉 온도가 높으면, distribution 이 완만하게 변하여 다양한 값들에 대하여 확률적으로 선택하게 됨 → 무작위성이 높아짐
  - `temperature` 값이 작아지면 distribution 이 특정 평균 값 주변으로 narrow하게 변하여 특정 값에 대하여 선택